In [ ]:
pip install delta-spark

In [ ]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType

In [ ]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/nba'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

In [18]:
spark.sql(
    """
    SHOW TABLES FROM nba
    """
).show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|     nba|  bz_abrev|      false|
|     nba|bz_players|      false|
+--------+----------+-----------+



In [17]:
spark.sql(
    """
    DROP TABLE IF EXISTS nba.bz_teams
    """
)

DataFrame[]

In [19]:
spark.sql(
    """
    CREATE EXTERNAL TABLE nba.bz_teams (
        league STRING,
        team STRING,
        wins INT,
        loses INT,
        win_lose_percentage DOUBLE,
        finish_position INT,
        srs DOUBLE,
        pace DOUBLE,
        real_pace DOUBLE,
        ortg DOUBLE,
        rel_ortg DOUBLE,
        drtg DOUBLE,
        rel_drtg DOUBLE,
        playoffs STRING,
        coach STRING,
        top_ws STRING
    )
    USING DELTA
    PARTITIONED BY(
    season STRING
    )
    TBLPROPERTIES (
   'skip.header.line.count'='1'
    )
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/bz_teams'
    """
)

21/11/30 00:29:26 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `nba`.`bz_teams` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [20]:
spark.sql(
    """
    SHOW TABLES FROM nba
    """
).show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|     nba|  bz_abrev|      false|
|     nba|bz_players|      false|
|     nba|  bz_teams|      false|
+--------+----------+-----------+



In [21]:
spark.sql(
    """
    SELECT * FROM nba.bz_teams
    """
).show()

+------+----+----+-----+-------------------+---------------+---+----+---------+----+--------+----+--------+--------+-----+------+------+
|league|team|wins|loses|win_lose_percentage|finish_position|srs|pace|real_pace|ortg|rel_ortg|drtg|rel_drtg|playoffs|coach|top_ws|season|
+------+----+----+-----+-------------------+---------------+---+----+---------+----+--------+----+--------+--------+-----+------+------+
+------+----+----+-----+-------------------+---------------+---+----+---------+----+--------+----+--------+--------+-----+------+------+



In [22]:
file_location = "hdfs://hdfs-nn:9000/projeto/triplos/bronze/nba_teams"

df = spark\
    .read\
    .format("csv")\
    .option('header','true')\
    .option("inferSchema" , "true")\
    .option("delimiter",",")\
    .load(file_location)

df.printSchema()
df.show()

root
 |-- Season: string (nullable = true)
 |-- Lg: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- W: integer (nullable = true)
 |-- L: integer (nullable = true)
 |-- W/L%: double (nullable = true)
 |-- Finish: integer (nullable = true)
 |-- SRS: double (nullable = true)
 |-- Pace: double (nullable = true)
 |-- Rel_Pace: double (nullable = true)
 |-- ORtg: double (nullable = true)
 |-- Rel_ORtg: double (nullable = true)
 |-- DRtg: double (nullable = true)
 |-- Rel_DRtg: double (nullable = true)
 |-- Playoffs: string (nullable = true)
 |-- Coaches: string (nullable = true)
 |-- Top WS: string (nullable = true)

+-------+---+---------------+---+---+-----+------+-----+----+--------+-----+--------+-----+--------+--------------------+--------------------+------------------+
| Season| Lg|           Team|  W|  L| W/L%|Finish|  SRS|Pace|Rel_Pace| ORtg|Rel_ORtg| DRtg|Rel_DRtg|            Playoffs|             Coaches|            Top WS|
+-------+---+---------------+---+---+--

In [56]:
df1 = df.withColumnRenamed("Top WS","TopWs")
df1.printSchema()

root
 |-- Season: string (nullable = true)
 |-- Lg: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- W: integer (nullable = true)
 |-- L: integer (nullable = true)
 |-- W/L%: double (nullable = true)
 |-- Finish: integer (nullable = true)
 |-- SRS: double (nullable = true)
 |-- Pace: double (nullable = true)
 |-- Rel_Pace: double (nullable = true)
 |-- ORtg: double (nullable = true)
 |-- Rel_ORtg: double (nullable = true)
 |-- DRtg: double (nullable = true)
 |-- Rel_DRtg: double (nullable = true)
 |-- Playoffs: string (nullable = true)
 |-- Coaches: string (nullable = true)
 |-- TopWs: string (nullable = true)



In [58]:
#write df to hive deltalake_table
#.select("customer_name","month", "sales", "year")  reorder columns to match parquet table sequence

df1 \
    .select("*") \
    .write \
    .mode("overwrite") \
    .partitionBy("season") \
    .format("delta") \
    .option("overwriteSchema", "true")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/bz_teams")

In [59]:
spark.sql(
    """
    SELECT * from nba.bz_teams
    """
).show()

+-------+---+--------------------+---+---+-----+------+-----+----+--------+-----+--------+-----+--------+--------------------+--------------------+------------------+
| Season| Lg|                Team|  W|  L| W/L%|Finish|  SRS|Pace|Rel_Pace| ORtg|Rel_ORtg| DRtg|Rel_DRtg|            Playoffs|             Coaches|             TopWs|
+-------+---+--------------------+---+---+-----+------+-----+----+--------+-----+--------+-----+--------+--------------------+--------------------+------------------+
|2008-09|NBA|     Boston Celtics*| 62| 20|0.756|     1| 7.44|90.4|    -1.3|110.5|     2.2|102.3|    -6.0| Lost E. Conf. Semis|   D. Rivers (62-20)|   R. Allen (11.1)|
|2008-09|NBA|     Toronto Raptors| 33| 49|0.402|     4|-2.54|92.0|     0.3|107.0|    -1.3|110.0|     1.7|                null|S. Mitchell (8-9)...|     C. Bosh (9.7)|
|2008-09|NBA|     New York Knicks| 32| 50| 0.39|     5|-2.33|96.7|     5.0|108.1|    -0.2|110.8|     2.5|                null| M. D'Antoni (32-50)|      D. Lee (9.0)

In [60]:
spark.sql(
    """
    DESCRIBE HISTORY nba.bz_teams
    """
).show()

#isto permite ver o historico de versoes da tabela

+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|           timestamp|userId|userName|   operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+--------------------+------+--------+------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      1|2021-11-30 00:43:...|  null|    null|       WRITE|{mode -> Overwrit...|null|    null|     null|          0|          null|        false|{numFiles -> 72, ...|        null|
|      0|2021-11-30 00:29:...|  null|    null|CREATE TABLE|{isManaged -> fal...|null|    null|     null|       null|          null|         true|                  {}|        null|
+-------+--------------------+------+--------+------------+--------------------+----+--------+------

In [ ]:
spark.stop()